# 1. Загрузка библиотек и данных

In [1]:
import pandas as pd
import numpy as np

In [2]:
!gdown 17SggAZPeOUvM68ZaZChdv_jQK3g9xUd9
!unzip kion_train.zip

Downloading...
From: https://drive.google.com/uc?id=17SggAZPeOUvM68ZaZChdv_jQK3g9xUd9
To: /content/kion_train.zip
100% 78.8M/78.8M [00:00<00:00, 253MB/s]
Archive:  kion_train.zip
replace kion_train/interactions.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: kion_train/interactions.csv  
  inflating: __MACOSX/kion_train/._interactions.csv  
  inflating: kion_train/users.csv    
  inflating: __MACOSX/kion_train/._users.csv  
  inflating: kion_train/items.csv    
  inflating: __MACOSX/kion_train/._items.csv  


In [3]:
df_1 = pd.read_csv('/content/kion_train/interactions.csv', parse_dates=['last_watch_dt'])
df_2 = pd.read_csv('/content/kion_train/items.csv')
df_3 = pd.read_csv('/content/kion_train/users.csv')

# 2. Краткий обзор данных

In [4]:
df_1.head(10)

,user_id,item_id,last_watch_dt,total_dur,watched_pct
0,176549,9506,2021-05-11,4250,72.0
1,699317,1659,2021-05-29,8317,100.0
2,656683,7107,2021-05-09,10,0.0
3,864613,7638,2021-07-05,14483,100.0
4,964868,9506,2021-04-30,6725,100.0
5,1032142,6686,2021-05-13,11286,100.0
6,1016458,354,2021-08-14,1672,25.0
7,884009,693,2021-08-04,703,14.0
8,648682,1449,2021-06-13,26246,75.0
9,203219,13582,2021-08-22,6975,100.0


*  Колонка **userId** - идентификаторы пользователей сайта kion;
*  Колонка **movieId** - идентификаторы фильмов;
*  Колонка **last_watch_dt** - дата просмотра;
*  Колонка **total_dur** - время просмотра в секундах;
*  Колонка **watched_pct** - просмотр в процентах.

Другие таблички нам пока не нужны. Их будем использовать в более сложных моделях для генерации признаков.

In [5]:
df_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5476251 entries, 0 to 5476250
Data columns (total 5 columns):
 #   Column         Dtype         
---  ------         -----         
 0   user_id        int64         
 1   item_id        int64         
 2   last_watch_dt  datetime64[ns]
 3   total_dur      int64         
 4   watched_pct    float64       
dtypes: datetime64[ns](1), float64(1), int64(3)
memory usage: 208.9 MB


In [6]:
df_1.apply("nunique")

user_id          962179
item_id           15706
last_watch_dt       163
total_dur        129788
watched_pct         101
dtype: int64

---

# 3. Обучение и предсказание модели

Будем рекомендовать всем пользователям самые популярные топ-10 за последние 14 дней.

In [7]:
def pop_14d():
  K = 10
  days = 14
  recommendations = []
  df_1 = pd.read_csv('/content/kion_train/interactions.csv', parse_dates=['last_watch_dt'])
  min_date = df_1['last_watch_dt'].max().normalize() - pd.DateOffset(days)
  recommendations = df_1.loc[df_1['last_watch_dt'] > min_date, 'item_id'].value_counts().head(K).index.values

  return list(recommendations)

In [8]:
pop_14d()

array([10440,  9728, 15297, 13865, 12192,   341,  4151,  3734, 12360,
        7793])